In [ ]:
pip install implicit

     |████████████████████████████████| 1.1MB 2.8MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3468807 sha256=583ba535164b3f518ddc2750eafdb55eec3877f74295608d087c28694692d99a
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import scipy
import implicit
import random
from datetime import datetime

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
data = pd.read_excel(url)
data.head(100)
#data.info()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
95,536378,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2010-12-01 09:37:00,2.55,14688.0,United Kingdom
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.42,14688.0,United Kingdom
97,536378,21975,PACK OF 60 DINOSAUR CAKE CASES,24,2010-12-01 09:37:00,0.55,14688.0,United Kingdom
98,536378,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,2010-12-01 09:37:00,0.55,14688.0,United Kingdom


In [ ]:
data = data[pd.notna(data['CustomerID'])]
#data.info()

In [ ]:
# to remove all the missing values from the dataframe. the rows are reduced from 541908 to 406829.
data.dropna() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [ ]:
# to remove all the negative values from the "Quantity" column.
#df.ix[] will slice the labels and integers and returns the dataframe as required.
#here we are slicing the rows(default) from quantity attribute whose values are less than zero.
data = data[data['Quantity']>0]
print(data)


       InvoiceNo StockCode  ... CustomerID         Country
0         536365    85123A  ...    17850.0  United Kingdom
1         536365     71053  ...    17850.0  United Kingdom
2         536365    84406B  ...    17850.0  United Kingdom
3         536365    84029G  ...    17850.0  United Kingdom
4         536365    84029E  ...    17850.0  United Kingdom
...          ...       ...  ...        ...             ...
541904    581587     22613  ...    12680.0          France
541905    581587     22899  ...    12680.0          France
541906    581587     23254  ...    12680.0          France
541907    581587     23255  ...    12680.0          France
541908    581587     22138  ...    12680.0          France

[397924 rows x 8 columns]


In [ ]:
recData = data.groupby(['CustomerID','StockCode','Description'], as_index = False).agg({'InvoiceNo': 'nunique'})
#'nunique' returns the number of unique data in the group.
recData

,CustomerID,StockCode,Description,InvoiceNo
0,12346.0,23166,MEDIUM CERAMIC TOP STORAGE JAR,1
1,12347.0,16008,SMALL FOLDING SCISSOR(POINTED EDGE),1
2,12347.0,17021,NAMASTE SWAGAT INCENSE,1
3,12347.0,20665,RED RETROSPOT PURSE,1
4,12347.0,20719,WOODLAND CHARLOTTE BAG,4
...,...,...,...,...
268403,18287.0,72351B,SET/6 PINK BUTTERFLY T-LIGHTS,1
268404,18287.0,84507C,BLUE CIRCLES DESIGN MONKEY DOLL,1
268405,18287.0,85039A,SET/4 RED MINI ROSE CANDLE IN BOWL,2
268406,18287.0,85039B,S/4 IVORY MINI ROSE CANDLE IN BOWL,2


In [ ]:

recData.rename(columns = {'InvoiceNo':'score'}, inplace = True)

In [ ]:
recData

,CustomerID,StockCode,Description,score
0,12346.0,23166,MEDIUM CERAMIC TOP STORAGE JAR,1
1,12347.0,16008,SMALL FOLDING SCISSOR(POINTED EDGE),1
2,12347.0,17021,NAMASTE SWAGAT INCENSE,1
3,12347.0,20665,RED RETROSPOT PURSE,1
4,12347.0,20719,WOODLAND CHARLOTTE BAG,4
...,...,...,...,...
268403,18287.0,72351B,SET/6 PINK BUTTERFLY T-LIGHTS,1
268404,18287.0,84507C,BLUE CIRCLES DESIGN MONKEY DOLL,1
268405,18287.0,85039A,SET/4 RED MINI ROSE CANDLE IN BOWL,2
268406,18287.0,85039B,S/4 IVORY MINI ROSE CANDLE IN BOWL,2


In [ ]:
recData.info

<bound method DataFrame.info of         CustomerID StockCode                          Description  score
0          12346.0     23166       MEDIUM CERAMIC TOP STORAGE JAR      1
1          12347.0     16008  SMALL FOLDING SCISSOR(POINTED EDGE)      1
2          12347.0     17021               NAMASTE SWAGAT INCENSE      1
3          12347.0     20665                 RED RETROSPOT PURSE       1
4          12347.0     20719               WOODLAND CHARLOTTE BAG      4
...            ...       ...                                  ...    ...
268403     18287.0    72351B       SET/6 PINK  BUTTERFLY T-LIGHTS      1
268404     18287.0    84507C      BLUE CIRCLES DESIGN MONKEY DOLL      1
268405     18287.0    85039A   SET/4 RED MINI ROSE CANDLE IN BOWL      2
268406     18287.0    85039B   S/4 IVORY MINI ROSE CANDLE IN BOWL      2
268407     18287.0    85040A      S/4 PINK FLOWER CANDLES IN BOWL      1

[268408 rows x 4 columns]>

In [ ]:
# we generate the pivot data to produce a utility matrix which will be further factorized based on its features like sparsity etc.
# in our pivot table we give the values to be the score that is the number of units the customer has purchased per item. 
#this will let us the interest of the customer on the item purcahsed
pivotData = pd.pivot_table(recData, values = 'score', index = 'CustomerID', columns = 'Description', aggfunc = np.sum)

In [ ]:

print(pivotData)


Description   4 PURPLE FLOCK DINNER CANDLES  ...  ZINC WIRE SWEETHEART LETTER TRAY
CustomerID                                   ...                                  
12346.0                                 NaN  ...                               NaN
12347.0                                 NaN  ...                               NaN
12348.0                                 NaN  ...                               NaN
12349.0                                 NaN  ...                               NaN
12350.0                                 NaN  ...                               NaN
...                                     ...  ...                               ...
18280.0                                 NaN  ...                               NaN
18281.0                                 NaN  ...                               NaN
18282.0                                 NaN  ...                               NaN
18283.0                                 NaN  ...                               NaN
1828

In [ ]:
#numpy.ma.masked_invalid is a function which masks all the invalid data from the pivot table. invalid data comprises data with Nan's,infs etc. \
#and then we compress the data. 
# to find the sparsity in the pivot table we use the following formula.
masked = np.ma.masked_invalid(pivotData)
mc = masked.compressed()
a = (1 - mc.shape[0]/np.prod(pivotData.shape))
print("sparsity is:", a*100, "%")
print(mc)


sparsity is: 98.40512324620475 %
[1. 5. 1. ... 1. 2. 1.]


In [ ]:
pivotData = pivotData.fillna(0)
pivotData

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,I LOVE LONDON MINI RUCKSACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,12 PENCILS TALL TUBE WOODLAND,12 PINK HEN+CHICKS IN BASKET,12 PINK ROSE PEG PLACE SETTINGS,12 RED ROSE PEG PLACE SETTINGS,15 PINK FLUFFY CHICKS IN BOX,15CM CHRISTMAS GLASS BALL 20 LIGHTS,16 PC CUTLERY SET PANTRY DESIGN,16 PIECE CUTLERY SET PANTRY DESIGN,18PC WOODEN CUTLERY SET DISPOSABLE,2 DAISIES HAIR COMB,2 PICTURE BOOK EGGS EASTER BUNNY,2 PICTURE BOOK EGGS EASTER CHICKS,2 PICTURE BOOK EGGS EASTER DUCKS,20 DOLLY PEGS RETROSPOT,200 BENDY SKULL STRAWS,...,YELLOW DRAGONFLY HELICOPTER,YELLOW EASTER EGG HUNT START POST,YELLOW FELT HANGING HEART W FLOWER,YELLOW FLOWERS FELT HANDBAG KIT,YELLOW GIANT GARDEN THERMOMETER,YELLOW METAL CHICKEN HEART,YELLOW PINK FLOWER DESIGN BIG BOWL,YELLOW POT PLANT CANDLE,YELLOW SHARK HELICOPTER,YELLOW/BLUE RETRO RADIO,YELLOW/ORANGE FLOWER DESIGN PLATE,YELLOW/PINK FLOWER DESIGN BIG MUG,YOU'RE CONFUSING ME METAL SIGN,YULETIDE IMAGES GIFT WRAP SET,YULETIDE IMAGES S/6 PAPER BOXES,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FINISH 15CM PLANTER POTS,ZINC FOLKART SLEIGH BELLS,ZINC HEART FLOWER T-LIGHT HOLDER,ZINC HEART LATTICE 2 WALL PLANTER,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC HEARTS PLANT POT HOLDER,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC PLANT POT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
CustomerID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
12346.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12347.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12348.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12349.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12350.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [ ]:
pivotData.shape

(4339, 3877)

In [ ]:
X_ = pivotData.values.T
np.all(np.isfinite(X_))
X_.shape


(3877, 4339)

In [ ]:
T = pivotData.T
np.all(np.isfinite(T))
print(T)

CustomerID                        12346.0  12347.0  ...  18283.0  18287.0
Description                                         ...                  
 4 PURPLE FLOCK DINNER CANDLES        0.0      0.0  ...      0.0      0.0
 50'S CHRISTMAS GIFT BAG LARGE        0.0      0.0  ...      0.0      0.0
 DOLLY GIRL BEAKER                    0.0      0.0  ...      0.0      0.0
 I LOVE LONDON MINI BACKPACK          0.0      0.0  ...      0.0      0.0
 I LOVE LONDON MINI RUCKSACK          0.0      0.0  ...      0.0      0.0
...                                   ...      ...  ...      ...      ...
ZINC T-LIGHT HOLDER STARS SMALL       0.0      0.0  ...      0.0      0.0
ZINC TOP  2 DOOR WOODEN SHELF         0.0      0.0  ...      0.0      0.0
ZINC WILLIE WINKIE  CANDLE STICK      0.0      0.0  ...      0.0      0.0
ZINC WIRE KITCHEN ORGANISER           0.0      0.0  ...      0.0      0.0
ZINC WIRE SWEETHEART LETTER TRAY      0.0      0.0  ...      0.0      0.0

[3877 rows x 4339 columns]


In [ ]:
from sklearn.preprocessing import StandardScaler
standard_data = StandardScaler().fit_transform(X_)
standard_data

array([[-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898],
       [-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898],
       [-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898],
       ...,
       [-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898],
       [-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898],
       [-0.01606231, -0.13880275, -0.06627249, ..., -0.05572061,
        -0.19462489, -0.11858898]])

In [ ]:
standard_data.shape

(3877, 4339)

In [ ]:
from numpy import zeros
from numpy import diag
from scipy.linalg import svd
#U - is a orthogonal matrix, s- is a diagnol matrix, VT is an orthogonal and  transposed matrix.
U,s,VT = svd(standard_data)
Sigma = zeros((standard_data.shape[0], standard_data.shape[1]))
Sigma[:standard_data.shape[0], :standard_data.shape[0]] = diag(s)
n_elements = 10
Sigma = Sigma[:, :n_elements]
VT = VT[:n_elements, :]
B = U.dot(Sigma.dot(VT))
print(B)
print("---------------------------------------------")
T = U.dot(Sigma)
print(T)
print("---------------------------------------------")
T = standard_data.dot(VT.T)
print(T)


[[-0.01210521 -0.06527048 -0.08100465 ... -0.0140713  -0.19951121
  -0.00740916]
 [-0.00200932  0.03920929  0.00399092 ...  0.00450629 -0.04357017
   0.09593092]
 [-0.01352989  0.28417708  0.17829646 ... -0.01082997  0.35373394
  -0.01175366]
 ...
 [ 0.01762124 -0.09310763  0.0761613  ...  0.06687664 -0.39172976
   0.0144467 ]
 [-0.00257449 -0.09229174 -0.07216501 ... -0.03819224 -0.2220128
  -0.03062086]
 [-0.00607603 -0.07504295 -0.11669014 ... -0.04716138 -0.18106858
  -0.02719578]]
---------------------------------------------
[[-4.43984505  1.06708654  1.26245359 ... -0.17840442  0.99466976
   0.45814904]
 [ 2.63371286 -0.17391007 -1.53702947 ... -3.8891881   1.62035746
  -0.38052925]
 [ 1.74756298 -5.8214428  -4.14844692 ... -0.50869279 -1.17534161
   2.66764865]
 ...
 [ 7.19576806  7.81737439 -0.76494797 ... -4.86098412 -1.88013394
  -2.99869509]
 [-6.27965818 -0.09284113  0.39475334 ...  0.31063362 -0.20232051
  -0.09467344]
 [-5.74474751 -0.1561442   1.11865925 ... -0.32532192

In [ ]:
from numpy import array
from sklearn.decomposition import TruncatedSVD
print(standard_data)
# svd
svd = TruncatedSVD(n_components=10)
svd.fit(standard_data)
result = svd.transform(standard_data)
print(result)

[[-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]
 [-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]
 [-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]
 ...
 [-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]
 [-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]
 [-0.01606231 -0.13880275 -0.06627249 ... -0.05572061 -0.19462489
  -0.11858898]]
[[-4.43982051  1.06570026 -1.25979244 ...  0.10437445  1.06265948
   0.4041703 ]
 [ 2.63365551 -0.17104877  1.52980433 ...  4.0677327   1.68119058
  -0.25618996]
 [ 1.74761169 -5.8229711   4.1633514  ...  0.28342435 -1.04837218
   1.75443186]
 ...
 [ 7.19573333  7.81997792  0.75016086 ...  5.03385796 -1.91988356
  -2.74297515]
 [-6.27966334 -0.09278609 -0.39628039 ... -0.28580984 -0.16880612
  -0.01260818]
 [-5.74475393 -0.15596241 -1.11937867 ...  0.32837558 -0.48697745
   0.32270679]]


In [ ]:
result.shape

(3877, 10)

In [ ]:
print(U)
print("---------------------------------------------")
print(s)
print("---------------------------------------------")
print(VT)

[[-5.82919651e-03  2.13142159e-03  2.76145058e-03 ... -4.54760356e-38
   8.94878990e-39 -8.15849154e-40]
 [ 3.45787514e-03 -3.47371709e-04 -3.36204908e-03 ...  1.27255448e-16
   1.21372992e-16  1.55723672e-16]
 [ 2.29442423e-03 -1.16278750e-02 -9.07418007e-03 ...  1.52283705e-16
   1.82728458e-16  2.08140137e-16]
 ...
 [ 9.44752477e-03  1.56145916e-02 -1.67322272e-03 ...  7.50132378e-17
   9.02598309e-17  1.88813808e-16]
 [-8.24473853e-03 -1.85442873e-04  8.63470818e-04 ...  1.64184631e-16
  -2.47306516e-16  5.60329235e-17]
 [-7.54243938e-03 -3.11885788e-04  2.44691945e-03 ...  3.51661822e-16
   1.31920299e-16  4.37529787e-16]]
---------------------------------------------
[7.61656437e+02 5.00645459e+02 4.57170443e+02 ... 5.45221397e-14
 5.45221397e-14 5.45221397e-14]
---------------------------------------------
[[ 0.00081137  0.01359452  0.01069201 ...  0.00912497  0.04004652
   0.00216078]
 [ 0.00052027  0.00090481 -0.02117833 ...  0.01433215 -0.06435626
   0.00015843]
 [-0.00141353

In [ ]:

print(U.shape)
print(s.shape)
print(VT.shape)

(3877, 3877)
(3877,)
(10, 4339)


In [ ]:
#print(np.linalg.matrix_rank(U))
#print(np.linalg.matrix_rank(s))
#print(np.linalg.matrix_rank(VT))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category = RuntimeWarning)
corr = np.corrcoef(result)
print(corr)
corr.shape


[[ 1.         -0.45875988 -0.47874457 ... -0.46113209  0.89984875
   0.90109396]
 [-0.45875988  1.          0.15591251 ...  0.41562562 -0.44447166
  -0.40449602]
 [-0.47874457  0.15591251  1.         ... -0.35081165 -0.32335134
  -0.36689663]
 ...
 [-0.46113209  0.41562562 -0.35081165 ...  1.         -0.43188243
  -0.40715993]
 [ 0.89984875 -0.44447166 -0.32335134 ... -0.43188243  1.
   0.97997203]
 [ 0.90109396 -0.40449602 -0.36689663 ... -0.40715993  0.97997203
   1.        ]]


(3877, 3877)

In [ ]:
decscription_columns = pivotData.columns
description_columns_list = list(decscription_columns)
test_recommendation = description_columns_list.index('NOVELTY BISCUITS CAKE STAND 3 TIER')
print(test_recommendation)

2159


In [ ]:
corr_test_recommendation  = corr[test_recommendation]
list(decscription_columns[(corr_test_recommendation>=0.8)])

['BAKING MOULD HEART MILK CHOCOLATE',
 'BLUE POLKADOT WASHING UP GLOVES',
 'CERAMIC CAKE DESIGN SPOTTED MUG',
 'CERAMIC CAKE STAND + HANGING CAKES',
 'CERAMIC STRAWBERRY DESIGN MUG',
 'FLOWER VINE RAFFIA FOOD COVER',
 'GLAMOROUS  MUG',
 'LARGE CAKE STAND  HANGING STRAWBERY',
 'MINI CAKE STAND  HANGING STRAWBERY',
 'MINI CAKE STAND WITH HANGING CAKES',
 'NOVELTY BISCUITS CAKE STAND 3 TIER',
 'RED RETROSPOT APRON ',
 'RED RETROSPOT OVEN GLOVE DOUBLE',
 'RED RETROSPOT TEA CUP AND SAUCER ',
 'RED RETROSPOT TRADITIONAL TEAPOT ',
 'SET 3 RETROSPOT TEA,COFFEE,SUGAR',
 'STRAWBERRY FAIRY CAKE TEAPOT',
 'SWEETHEART CERAMIC TRINKET BOX',
 'TEA TIME OVEN GLOVE']

In [ ]:

#decscription_columns1 = T.columns
#description_columns_list1 = list(decscription_columns1)
#test_recommendation1 = description_columns_list1.index(12348.0)
#print(test_recommendation1)

generating recommendations using implicit



In [ ]:
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from pandas.api.types import CategoricalDtype


In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data_cleaned = data.loc[pd.isnull(data.CustomerID)== False ]

In [ ]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    397924 non-null  object        
 1   StockCode    397924 non-null  object        
 2   Description  397924 non-null  object        
 3   Quantity     397924 non-null  int64         
 4   InvoiceDate  397924 non-null  datetime64[ns]
 5   UnitPrice    397924 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      397924 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.3+ MB


In [ ]:
items_check = data_cleaned[['StockCode','Description']].drop_duplicates()
items_check['StockCode'] = data_cleaned.StockCode.astype(str)

In [ ]:
items_check.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [ ]:
data_cleaned['CustomerID'] = data_cleaned.CustomerID.astype(int)
data_cleaned = data_cleaned[['StockCode', 'Quantity', 'CustomerID']] 
data_cleaned_grouped = data_cleaned.groupby(['CustomerID', 'StockCode']).sum().reset_index()
data_cleaned_grouped.Quantity.loc[data_cleaned_grouped.Quantity == 0] == 1
grouped_purchased_data = data_cleaned_grouped.query('Quantity > 0')

In [ ]:
grouped_purchased_data.head()

,CustomerID,StockCode,Quantity
0,12346,23166,74215
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [ ]:
customers = list(np.sort(grouped_purchased_data.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased_data.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased_data.Quantity) # All of our purchases

rows = grouped_purchased_data.CustomerID.astype(CategoricalDtype(categories = customers)).cat.codes 
# Get the associated row indices
cols = grouped_purchased_data.StockCode.astype(CategoricalDtype(categories =  products)).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

In [ ]:
purchases_sparse

<4339x3665 sparse matrix of type '<class 'numpy.longlong'>'
	with 266802 stored elements in Compressed Sparse Row format>

In [ ]:
#print(purchases_sparse)

In [ ]:
print(len(purchases_sparse.nonzero()[1]))

266802


In [ ]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.32225693738097

In [ ]:
import random

In [ ]:
def train_data(ratings, pct_test = 0.2): 
  test_set = ratings.copy()
  test_set[test_set != 0 ] = 1
  train_set = ratings.copy()
  nonzero_inds = train_set.nonzero()
  nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1]))
  random.seed(0)
  num_samples = int(np.ceil(pct_test*len(nonzero_pairs)))
  samples = random.sample(nonzero_pairs, num_samples)
  user_inds = [index[0] for index in samples]
  item_inds = [index[1] for index in samples] # Get the item column indices
  train_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
  train_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
  return train_set, test_set, list(set(user_inds)) 




In [ ]:
product_train_test, product_test_test, product_users_altered_data = train_data(purchases_sparse, pct_test = 0.2)


In [ ]:
def implicit_als(training_set, lambda_value = 0.1, alpha = 40, iterations = 10, rank_size = 20, seed=0):
  conf = (alpha*training_set)

  num_users = conf.shape[0]
  num_items = conf.shape[1]

  rstate = np.random.RandomState(seed)
  X = sparse.csr_matrix(rstate.normal(size = (num_users, rank_size))) # Random numbers in a m x rank shape
  Y = sparse.csr_matrix(rstate.normal(size = (num_items, rank_size))) # Normally this would be rank x n but we can 
                                                                 # transpose at the end. Makes calculation more simple.
  X_eye = sparse.eye(num_users)
  Y_eye = sparse.eye(num_items)
  lambda_eye = lambda_value * sparse.eye(rank_size)


  for iter_step in range(iterations):

    xTx = X.T.dot(X)
    yTy = Y.T.dot(Y)

    for u in range(num_users):
      conf_sample = conf[u,:].toarray()
      pref = conf_sample.copy()
      pref[pref!=0] = 1
      CuI = sparse.diags(conf_sample,[0])
      yTCuIY = Y.T.dot(CuI).dot(Y)
      yTCupu = Y.T.dot(CuI + Y_eye).dot(pref.T)
      
      X[u] = spsolve(yTy+yTCuIY+lambda_eye, yTCupu)

    for i in range(num_items):
      conf_samp = conf[:,i].T.toarray() # transpose to get it in row format and convert to dense
      pref = conf_samp.copy()
      pref[pref != 0] = 1 # Create binarized preference vector
      CiI = sparse.diags(conf_samp, [0]) # Get Ci - I term, don't need to subtract 1 since we never added it
      xTCiIX = X.T.dot(CiI).dot(X) # This is the xT(Cu-I)X term
      xTCiPi = X.T.dot(CiI + X_eye).dot(pref.T) # This is the xTCiPi term
      Y[i] = spsolve(xTx + xTCiIX + lambda_eye, xTCiPi)
      # Solve for Yi = ((xTx + xT(Cu-I)X) + lambda*I)^-1)xTCiPi, equation 5 from the paper
      # End iterations
  return X, Y.T # Transpose at the end to make up for not being transposed at the beginning. 
                    # Y needs to be rank x n. Keep these as separate matrices for scale reasons. 



In [ ]:
user_vecs, item_vecs = implicit_als(product_train_test,lambda_value = 0.1, alpha = 15,iterations = 1, rank_size = 20)

In [ ]:
 user_vecs[0,:].dot(item_vecs).toarray()[0,:5]


array([ 0.9434495 , -0.0325035 , -0.09743154, -0.01528708, -0.33619334])

# speeding up **ALS**

In [ ]:
!pip install implicit

In [ ]:
alpha = 15
user_vecs, item_vecs = implicit.alternating_least_squares((product_train_test*alpha).astype('double'),factors = 20, regularization = 0.1, iterations = 50 )

This method is deprecated. Please use the AlternatingLeastSquares class instead
GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


# evaluating the **recommender** **system**

In [ ]:
from sklearn import metrics

In [ ]:
def area_under_curve_score(predictions, test):
  '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
  fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
  return metrics.auc(fpr, tpr)


In [ ]:
def calc_mean_areaUnderCurve(training_set, altered_users, predictions, test_set):
  store_auc = []
  popularity_auc = []
  pop_items = np.array(test_set.sum(axis = 0)).reshape(-1) # Get sum of item iteractions to find most popular
  item_vecs = predictions[1]
  for user in altered_users: # Iterate through each user that had an item altered
      training_row = training_set[user,:].toarray().reshape(-1) # Get the training set row
      zero_inds = np.where(training_row == 0) # Find where the interaction had not yet occurred
      # Get the predicted values based on our user/item vectors
      user_vec = predictions[0][user,:]
      pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
      # Get only the items that were originally zero
      # Select all ratings from the MF prediction for this user that originally had no iteraction
      actual = test_set[user,:].toarray()[0,zero_inds].reshape(-1) 
      # Select the binarized yes/no interaction pairs from the original full data
      # that align with the same pairs in training 
      pop = pop_items[zero_inds] # Get the item popularity for our chosen items
      store_auc.append(area_under_curve_score(pred, actual)) # Calculate AUC for the given user and store
      popularity_auc.append(area_under_curve_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
  return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark
      


In [ ]:
calc_mean_areaUnderCurve(product_train_test, product_users_altered_data,[sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test_test)
# AUC for our recommender system

(0.872, 0.816)

In [ ]:
customers_arr = np.array(customers) # Array of customer IDs from the ratings matrix
products_arr = np.array(products) # Array of product IDs from the ratings matrix

In [ ]:
def get_items_purchased(customer_id, mf_train, customers_list, products_list, item_lookup):
    '''
    This just tells me which items have been already purchased by a specific user in the training set. 
    
    parameters: 
    
    customer_id - Input the customer's id number that you want to see prior purchases of at least once
    
    mf_train - The initial ratings training set used (without weights applied)
    
    customers_list - The array of customers used in the ratings matrix
    
    products_list - The array of products used in the ratings matrix
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    returns:
    
    A list of item IDs and item descriptions for a particular customer that were already purchased in the training set
    '''
    cust_ind = np.where(customers_list == customer_id)[00] # Returns the index row of our customer id
    purchased_ind = mf_train[cust_ind,:].nonzero()[1] # Get column indices of purchased items
    prod_codes = products_list[purchased_ind] # Get the stock codes for our purchased items
    return item_lookup.loc[item_lookup.StockCode.isin(prod_codes)]

In [ ]:
customers_arr[:5]


array([12346, 12347, 12348, 12349, 12350])

In [ ]:
get_items_purchased(12346, product_train_test
                    , customers_arr, products_arr, items_check)


,StockCode,Description
61619,23166,MEDIUM CERAMIC TOP STORAGE JAR


In [ ]:
from sklearn.preprocessing import MinMaxScaler


In [ ]:
def rec_items(customer_id, mf_train, user_vecs, item_vecs, customer_list, item_list, item_lookup, num_items = 10):
    '''
    This function will return the top recommended items to our users 
    
    parameters:
    
    customer_id - Input the customer's id number that you want to get recommendations for
    
    mf_train - The training matrix you used for matrix factorization fitting
    
    user_vecs - the user vectors from your fitted matrix factorization
    
    item_vecs - the item vectors from your fitted matrix factorization
    
    customer_list - an array of the customer's ID numbers that make up the rows of your ratings matrix 
                    (in order of matrix)
    
    item_list - an array of the products that make up the columns of your ratings matrix
                    (in order of matrix)
    
    item_lookup - A simple pandas dataframe of the unique product ID/product descriptions available
    
    num_items - The number of items you want to recommend in order of best recommendations. Default is 10. 
    
    returns:
    
    - The top n recommendations chosen based on the user/item vectors for items never interacted with/purchased
    '''
    
    cust_ind = np.where(customer_list == customer_id)[0][0] # Returns the index row of our customer id, to find the index of value we use numpy.where as there exists no numpy.index
    pref_vec = mf_train[cust_ind,:].toarray() # Get the ratings from the training set ratings matrix
    pref_vec = pref_vec.reshape(-1) + 1 # Add 1 to everything, so that items not purchased yet become equal to 1
    pref_vec[pref_vec > 1] = 0 # Make everything already purchased zero
    rec_vector = user_vecs[cust_ind,:].dot(item_vecs.T) # Get dot product of user vector and all item vectors
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0] 
    recommend_vector = pref_vec*rec_vector_scaled 
    # Items already purchased have their recommendation multiplied by zero
    product_idx = np.argsort(recommend_vector)[::-1][:num_items] # Sort the indices of the items into order 
    # of best recommendations
    rec_list = [] # start empty list to store items
    for index in product_idx:
        code = item_list[index]
        rec_list.append([code, item_lookup.Description.loc[item_lookup.StockCode == code].iloc[0]]) 
        # Append our descriptions to the list
    codes = [item[0] for item in rec_list]
    descriptions = [item[1] for item in rec_list]
    final_frame = pd.DataFrame({'StockCode': codes, 'Description': descriptions}) # Create a dataframe 
    return final_frame[['StockCode', 'Description']] # Switch order of columns around

In [ ]:
rec_items(12346, product_train_test, user_vecs, item_vecs, customers_arr, products_arr, items_check,
                       num_items = 10)

,StockCode,Description
0,23167,SMALL CERAMIC TOP STORAGE JAR
1,23165,LARGE CERAMIC TOP STORAGE JAR
2,84826,ASSTD DESIGN 3D PAPER STICKERS
3,21390,FILIGRIS HEART WITH BUTTERFLY
4,23014,GLASS APOTHECARY BOTTLE ELIXIR
5,85061W,WHITE JEWELLED HEART DECORATION
6,23169,CLASSIC GLASS SWEET JAR
7,23418,LAVENDER TOILETTE BOTTLE
8,22665,RECIPE BOX BLUE SKETCHBOOK DESIGN
9,21917,SET 12 KIDS WHITE CHALK STICKS


In [ ]:
get_items_purchased(12353, product_train_test, customers_arr, products_arr, items_check)


,StockCode,Description
2148,37446,MINI CAKE STAND WITH HANGING CAKES
2149,37449,CERAMIC CAKE STAND + HANGING CAKES
4859,37450,CERAMIC CAKE BOWL + HANGING CAKES
5108,22890,NOVELTY BISCUITS CAKE STAND 3 TIER


In [ ]:
rec_items(12353, product_train_test, user_vecs, item_vecs, customers_arr, products_arr,
          items_check,num_items = 10)

,StockCode,Description
0,37447,CERAMIC CAKE DESIGN SPOTTED PLATE
1,22055,MINI CAKE STAND HANGING STRAWBERY
2,21231,SWEETHEART CERAMIC TRINKET BOX
3,21232,STRAWBERRY CERAMIC TRINKET BOX
4,22646,CERAMIC STRAWBERRY CAKE MONEY BANK
5,22649,STRAWBERRY FAIRY CAKE TEAPOT
6,22064,PINK DOUGHNUT TRINKET POT
7,22057,CERAMIC PLATE STRAWBERRY DESIGN
8,47559B,TEA TIME OVEN GLOVE
9,22063,CERAMIC BOWL WITH STRAWBERRY DESIGN


In [ ]:
get_items_purchased(12361, product_train_test, customers_arr, products_arr, items_check)


,StockCode,Description
34,22326,ROUND SNACK BOXES SET OF4 WOODLAND
35,22629,SPACEBOY LUNCH BOX
37,22631,CIRCUS PARADE LUNCH BOX
93,20725,LUNCH BAG RED RETROSPOT
369,22382,LUNCH BAG SPACEBOY DESIGN
547,22328,ROUND SNACK BOXES SET OF 4 FRUITS
549,22630,DOLLY GIRL LUNCH BOX
1241,22555,PLASTERS IN TIN STRONGMAN
58132,20725,LUNCH BAG RED SPOTTY


In [ ]:
rec_items(12361, product_train_test, user_vecs, item_vecs, customers_arr, products_arr, items_check,
                       num_items = 10)

,StockCode,Description
0,22551,PLASTERS IN TIN SPACEBOY
1,22662,LUNCH BAG DOLLY GIRL DESIGN
2,22383,LUNCH BAG SUKI DESIGN
3,20726,LUNCH BAG WOODLAND
4,20727,LUNCH BAG BLACK SKULL.
5,22554,PLASTERS IN TIN WOODLAND ANIMALS
6,23206,LUNCH BAG APPLE DESIGN
7,20728,LUNCH BAG CARS BLUE
8,23209,LUNCH BAG DOILEY PATTERN
9,20719,WOODLAND CHARLOTTE BAG


In [ ]:
print(customers_arr)

[12346 12347 12348 ... 18282 18283 18287]


In [ ]:
customer_id = 12348
c = np.where(customers_arr == customer_id)[0][0]